## Testing notebook for beta reconstruction software

For testing use the latest version of defdap on the grain_boundaies branch

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

from defdap.quat import Quat
import defdap.ebsd as ebsd

from defdap.plotting import PolePlot, MapPlot

from beta_reconstruction.reconstruction import do_reconstruction

%matplotlib qt

## Load in EBSD file, find grains and calculate grain average orientaions

In [2]:
dropboxPath = "/Users/mbcx9ma4/Dropbox (The University of Manchester)/" \
              "Projects/Beta Reconstruction/Beta Reconstruction/"
EbsdFilePath = dropboxPath + "data/triple_point/ZrNb_triplepoint_alpha"
# EbsdFilePath = dropboxPath + "data/hot_rolled/ZrNb_hot_rolled_alpha"

EbsdMap = ebsd.Map(EbsdFilePath, "hexagonal")
EbsdMap.buildQuatArray()

# Kuwahara filter - this will be slow.
# EbsdMap.filterData(misOriTol=5)

EbsdMap.findBoundaries(boundDef=3)
EbsdMap.findGrains(minGrainSize=3)

EbsdMap.calcGrainAvOris()

EbsdMap.buildNeighbourNetwork()

Loaded EBSD data (dimensions: 285 x 276 pixels, step size: 5.0 um)
Starting building quaternion array..

/Users/mbcx9ma4/modelling_stuff/packages/DataAnalysisUtilities/defdap/ebsd.py:185: UserWarning: No c/a ratio given. Using ideal ratio 1.633
  warnings.warn("No c/a ratio given. Using ideal ratio 1.633")


Finished building quaternion array           
Finished finding grain boundaries           
Finished finding grains           
Finished calculating grain mean orientations           
Finished constructing neighbour network           


## Calculate possible beta orientations from misorientations between each grain adn its neighbours

In [4]:
start = time.time()

do_reconstruction(EbsdMap, burg_tol=5., ori_tol=3.)

end = time.time()
print("{:.2f} minutes".format((end - start) / 60))

 Done 99 %3.76 minutes


## Look at the data stored for each grain

In [5]:
EbsdMap.locateGrainID()

In [6]:
grain = EbsdMap[8]

In [7]:
grain.betaOris

[[0.8226, -0.0830, -0.2905, 0.4818],
 [0.1826, -0.8403, -0.3539, 0.3679],
 [0.5803, -0.5330, -0.3720, 0.4906],
 [0.4663, -0.2672, -0.0019, -0.8433],
 [0.3979, -0.3645, 0.8010, -0.2592],
 [0.0684, 0.0973, -0.8029, -0.5841]]

In [8]:
grain.possibleBetaOris

[[[0.8134, -0.1007, -0.3073, 0.4835], [0.3984, -0.3679, 0.8069, -0.2340]],
 [[0.2483, 0.9227, 0.2552, -0.1481], [0.2999, -0.0815, -0.4747, -0.8234]],
 [[0.3934, -0.3702, 0.7943, -0.2780], [0.8329, -0.0709, -0.2803, 0.4720]],
 [[0.2962, -0.0831, -0.4748, -0.8246], [0.2498, 0.9212, 0.2571, -0.1512]],
 [[0.8227, -0.0917, -0.2903, 0.4801], [0.3919, -0.3654, 0.8051, -0.2543]],
 [[0.3005, -0.0910, -0.4803, -0.8190], [0.2481, 0.9236, 0.2469, -0.1564]],
 [[0.3016, -0.0853, -0.4732, -0.8233], [0.2513, 0.9227, 0.2519, -0.1484]],
 [[0.3126, -0.1069, -0.4705, -0.8182]],
 [[0.3114, -0.0882, -0.4590, -0.8274], [0.2636, 0.9225, 0.2470, -0.1362]],
 [[0.3084, -0.0925, -0.4741, -0.8195], [0.2528, 0.9249, 0.2417, -0.1491]],
 [[0.8161, -0.0872, -0.2991, 0.4867], [0.4033, -0.3629, 0.8025, -0.2481]],
 [[0.3977, -0.3608, 0.8044, -0.2542], [0.8195, -0.0879, -0.2911, 0.4857]],
 [[0.3084, -0.1056, -0.4660, -0.8225], [0.2668, 0.9220, 0.2359, -0.1520]],
 [[0.3085, -0.1047, -0.4699, -0.8204], [0.2627, 0.9231, 0.23

In [9]:
np.array(grain.betaDeviations) *180 /np.pi

array([3.83572963, 4.10000054, 4.2009164 , 4.20035926, 0.99954059,
       3.30822677, 3.38247078, 3.41887741, 2.69137772, 2.70480538,
       0.83630896, 0.75989446, 2.47811833, 2.71362684, 3.18845622,
       1.28800042, 3.62070255, 2.60705801, 3.19543254, 1.19537458,
       0.69126409, 0.54919711, 0.79435382, 0.65064293, 1.50882467,
       1.168161  , 2.13098647, 1.60098689, 2.67826334, 2.83496826,
       2.51133107, 3.62638061, 2.46810384, 0.80230785, 3.19859526,
       4.19370934, 3.27004932, 3.90097166, 2.26971135, 2.00137583,
       2.60748392, 3.24579266, 1.66761906, 1.95919824, 3.10854725,
       4.94408628, 3.64661231, 4.28921174, 3.08690819, 3.12241564,
       2.31075372, 3.29073666, 0.79202856, 2.16887027, 2.17198667,
       4.03221838])

In [10]:
grain.variantCount

[42, 0, 0, 0, 51, 0]

burgers tol for each variant count - this will have to wait until we rewrtite the variant counting code

In [11]:
possibleBetaOris = [item for sublist in grain.possibleBetaOris for item in sublist]

directions = [
    np.array([1,0,0]), 
    np.array([0,1,0]), 
    np.array([0,0,1])
]
markerSize = 100

fig, axes = plt.subplots(1, len(directions))

for direction, ax in zip(directions, axes):
    plot = Quat.plotIPF(grain.betaOris, direction, "cubic", marker='o', s=markerSize, fig=fig, ax=ax)
    Quat.plotIPF(possibleBetaOris, direction, "cubic", s=markerSize, plot=plot)

fig.tight_layout()

## Find the most common variant for each grain and set this as the beta orientaion

In [12]:
modeVariants = []
parentBetaOris = []

for grain in EbsdMap:
#     modeVariant = np.argmax(grain.variantCount)
#     parentBetaOri = grain.betaOris[modeVariant]
    
    variantCount = np.array(grain.variantCount)
    # argmax would only pass one value if multiple variants have the same number of counts
    modeVariant = np.where(variantCount == np.max(variantCount))[0]
    if len(modeVariant) == 1:
        modeVariant = modeVariant[0]
        parentBetaOri = grain.betaOris[modeVariant]
    else:
        # multiple variants with same max
        modeVariant = -1
        parentBetaOri = Quat(1., 0., 0., 0.)
    
    modeVariants.append(modeVariant)
    grain.modeVariant = modeVariant
    parentBetaOris.append(parentBetaOri)
    grain.parentBetaOri = parentBetaOri

## Plot

### Vaiant number 

In [13]:
plot = EbsdMap.plotGrainDataMap(grainData=modeVariants, vmin=-1, vmax=5, cmap="Set1")
plot.addColourBar("mode variant")

### IPF of grains filled with average orientation

In [18]:
directions = [
    np.array([1,0,0]),
    np.array([0,1,0]),
    np.array([0,0,1])
]

fig, axes = plt.subplots(1, len(directions), figsize=(15,5))

for ax, direction in zip(axes, directions):
    betaGrainIPFColours = Quat.calcIPFcolours(parentBetaOris, direction, "cubic")

    EbsdMap.plotGrainDataMap(grainData=betaGrainIPFColours, fig=fig, ax=ax)
    
fig.tight_layout()

### Calculate and then plot beta orientation map

In [15]:
from beta_reconstruction.crystal_relations import unq_hex_syms, burg_trans

transformations = []
for sym in unq_hex_syms:
    transformations.append(burg_trans * sym.conjugate)
    
variantMap = EbsdMap.grainDataToMapData(modeVariants, bg=-2)
    
betaQuatArray = np.empty_like(EbsdMap.quatArray)
for i in range(EbsdMap.yDim):
    for j in range(EbsdMap.xDim):
        variant = variantMap[i, j]
        if variant < 0:
            # points not part of a grain (-2) and 
            # those that were not reconstructed (-1)
            betaQuatArray[i, j] = Quat(1, 0, 0, 0)
        else:
#             betaQuatArray[i, j] = burg_trans * unq_hex_syms[variant].conjugate * EbsdMap.quatArray[i, j]
            betaQuatArray[i, j] = transformations[variant] * EbsdMap.quatArray[i, j]

In [17]:
directions = [
    np.array([1,0,0]),
    np.array([0,1,0]),
    np.array([0,0,1])
]

fig, axes = plt.subplots(1, len(directions), figsize=(15,5))

for ax, direction in zip(axes, directions):
    betaIPFColours = Quat.calcIPFcolours(betaQuatArray.flatten(), direction, "cubic")
    # reshape back to map shape array
    betaIPFColours = np.reshape(betaIPFColours, (EbsdMap.yDim, EbsdMap.xDim, 3))
    
    # recolour the -1 and -2 variants
    # -1 grains not reconstructed (white)
    # -2 clusters too small to be a grain (black)
    indxs = np.where(variantMap == -1)
    betaIPFColours[indxs[0], indxs[1]] = np.array([1, 1, 1])
    indxs = np.where(variantMap == -2)
    betaIPFColours[indxs[0], indxs[1]] = np.array([0, 0, 0])
    
    plot = MapPlot.create(EbsdMap, betaIPFColours, fig=fig, ax=ax)
    
fig.tight_layout()